# 4.7 피봇테이블과 그룹 분석

## 연습문제 4.7.1
key1의 값을 기준으로 data1의 값을 분류하여 합계를 구한 결과를 시리즈가 아닌 데이터프레임으로 구한다.



In [32]:
import pandas as pd

df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]
})

# key1 값을 기준으로 data1의 값을 문류하여 합계를 구한 데이터프레임 결과
# 정답 1
df2.groupby(df2.key1).sum()[['data1']]
# 정답 2
df2.groupby(df2.key1).apply(lambda x : x.max())

,key1,key2,data1,data2
key1,,,,
A,A,two,5,50
B,B,two,4,40


## 연습 문제 4.7.2
붓꽃(iris) 데이터에서 붓꽃 종(species)별로 꽃잎길이(sepal_length), 꽃잎폭(sepal_width) 등의 평균을 구하라. 만약 붓꽃 종(species)이 표시되지 않았을 때 이 수치들을 이용하여 붓꽃 종을 찾아낼 수 있을지 생각하라.



In [44]:
# 종별로 각 열의 평균 구하기
iris.groupby(iris.species).mean()

# 평균 값에 중복이 없기 때문에 종을 찾아낼 수 있을 것 같다.

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.006,3.428,1.462,0.246
versicolor,5.936,2.770,4.260,1.326
virginica,6.588,2.974,5.552,2.026


## 연습문제 4.7.3
1. 팁의 비율이 요일과 점심/저녁 여부, 인원수에 어떤 영향을 받는지 살펴본다.

2. 어떤 요인이 가장 크게 작용하는지 판단할 수 있는 방법이 있는가?

In [107]:
import seaborn as sns

tips = sns.load_dataset('tips')

tips['tip_pct'] = tips['tip'] / tips['total_bill']



# 각 요인에 따른 tip_pct 출력
print(tips.pivot_table('tip_pct','day'))
print(tips.pivot_table('tip_pct','time'))
print(tips.pivot_table('tip_pct','size'))

# 각 요인에 따른 tip_pct 표준 편차 출력
print(tips.pivot_table('tip_pct','day').std())
print(tips.pivot_table('tip_pct','time').std())
print(tips.pivot_table('tip_pct','size').std())

# 표준편차가 클수록 값이 넓게 퍼져있어 요인에 따라 영향이 크다고 생각해서 표준편차를 비교해보았다.
# 그 결과 size 의 표준편차가 가장 커서 인원수가 팁의 비율에 큰 영향을 준다고 생각한다.

       tip_pct
day           
Thur  0.161276
Fri   0.169913
Sat   0.153152
Sun   0.166897
         tip_pct
time            
Lunch   0.164128
Dinner  0.159518
       tip_pct
size          
1     0.217292
2     0.165719
3     0.152157
4     0.145949
5     0.141495
6     0.156229
tip_pct    0.007366
dtype: float64
tip_pct    0.00326
dtype: float64
tip_pct    0.027826
dtype: float64


## 연습문제 4.7.4

타이타닉 승객 데이터를 이용하여 다음 분석을 실시하라. 데이터는 다음과 같이 받을 수 있다.

titanic = sns.load_dataset("titanic")

1. qcut 명령으로 세 개의 나이 그룹을 만든다.

2. 성별, 선실, 나이 그룹에 의한 생존율을 데이터프레임으로 계산한다. 행에는 성별 및 나이 그룹에 대한 다중 인덱스를 사용하고 열에는 선실 인덱스를 사용한다. 생존률은 해당 그룹의 생존 인원수를 전체 인원수로 나눈 값이다.

3. 성별 및 선실에 의한 생존율을 피봇 데이터 형태로 만든다.

In [196]:
import seaborn as sns

titanic = sns.load_dataset('titanic')

# 1번 문제
# qcut 명렬으로 세 개의 나이 그룹 만들기
# 세 개의 나이 그룹으로 나누는 함수 정의
def q3cut(s):
    return pd.qcut(s, 3, labels = ['노년', '중년', '소년']).astype(str)
# 함수를 사용하여 나이를 기준으로 세 개의 나이 그룹으로 나누기
titanic['age_group'] = titanic['age'].transform(q3cut)

# 2번 문제
# 생존률을 계산하는 함수에 s에 들어오는 인수가 시리즈라는 것에 유의해야한다.
def survive_rate(s):
    count = 0
    for i in s:
        if i == 1:
            count += 1
    return count / s.count()

# 열 인덱스를 class로 해주기 위해서는 행 인덱스로 설정한 후 unstack()으로 열 인덱스로 변경해준다.
titanic.groupby(['sex','age_group','class'])[['survived']].agg(survive_rate).unstack()

# 3번 문제
# 성별 및 선실에 의한 생존률을 피봇 데이터 형태로 만들기
titanic.pivot_table('survived','sex','class')


class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447
